In [1]:
library(MatchIt)
library(dplyr)
library(tidyr)
library(sparklyr)
library(ggplot2)
library(ggridges)
library(gridExtra)
library(Hmisc)
require(tidyverse)
require(lmerTest)
require(grid)
library(effsize)
library(egg)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: lattice

Loading required package: survival

Loading required package: Formula


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ gridExtra::combine() masks dplyr::combine()
✖ dplyr::filter()      masks stats::filter()
✖ purrr::invoke()  

In [2]:
VARIABLES_CLEAR_NAMES <- list (
  "vm1"= "Heart rate (HR)",
  "vm2"= "Temperature (central)",
  "vm3"= "Systolic arterial blood pressure (SBP)",
  "vm4"= "DBP",
  "vm5"= "Mean arterial blood pressure (MAP)",
  "vm9"= "Mean pulmonary arterial pressure (PAPm)",
  "vm12"= "PCWP",
  "vm13"= "Cardiac output (CO)",
  "vm14"= "Mixed venous oxygen saturation (SvO2)",
  "vm15"= "Central venous pressure (CVP)",
  "vm20"= "SpO2",
  "vm22"= "respiratory rate",
  "vm24"= "Urine",
  "pm39"= "Norepinephrine",
  "pm40"= "Epinephrine",
  "pm41"= "Dobutamine",
  "pm42"= "Milrinone",
  "pm43"= "Levosimendan",
  "pm44"= "Theophyllin",
  "pm45"= "Vasopressin",
  "pm46"= "Desmopressin",
  "vm263"= "Rekap time",
  "vm136"= "art. Lactate",
  "vm132"= "art. Base Excess",
  "vm134"= "art. Haemoglobin",
  "vm174"= "Glucose",
  "vm142"= "ScvO2",
  "vm131"= "Weight",
  "vm176"= "CRP",
  "vm161"= "ALAT",
  "vm138"= "art. pH",
  "vm140"= "a_PO2",
  "vm141"= "a_SO2",
  "vis"= "Vasoactive inotropic score (VIS)",
  "vbpip"= "MAP/VIS",
  "do2"= "dO2 = CO * Hb * a_So2 * 1.34",
  "perfpressure"= "Perfusion pressure",
  "svbpip"= "Perfussion pressure/VIS",
  "sv"= "Stroke volume (SV)",
  "svr"= "Systemic vascular resistance (SVR)"
)

VARIABLES_UNITS <- list (
  "vm1"= "\'/min",
  "vm2"= "°C",
  "vm3"= "mmHg",
  "vm4"= "mmHg",
  "vm5"= "mmHg",
  "vm9"= "mmHg",
  "vm12"= "mmHg",
  "vm13"= "l/min",
  "vm14"= "%",
  "vm15"= "mmHg",
  "vm20"= "%",
  "vm22"= "\'/min",
  "vm24"= "ml/h",
  "pm39"= "ug/min",
  "pm40"= "ug/min",
  "pm41"= "Fix unit",
  "pm42"= "Fix unit",
  "pm43"= "Fix unit",
  "pm44"= "Fix unit",
  "pm45"= "Fix unit",
  "pm46"= "Fix unit",
  "vm263"= "s",
  "vm136"= "mmol/l",
  "vm132"= "mmol/l",
  "vm134"= "g/l",
  "vm174"= "mmol/l",
  "vm142"= "%",
  "vm131"= "kg",
  "vm176"= "mg/l",
  "vm161"= "U/l",
  "vm138"= "pH",
  "vm140"= "mmHg",
  "vm141"= "%",
  "vis"= "VIS",
  "vbpip"= "mmHg/VIS",
  "do2"= "ml/min",
  "perfpressure"= "mmHg",
  "svbpip"= "mmHg/VIS",
  "sv"= "ml",
  "svr"="cm")

VARIABLES_YRANGE <- list (
  "vm1"= c(0,110),
  "vm2"= c(0,45),
  "vm3"= c(0,130),
  "vm4"= c(0,75),
  "vm5"= c(0,90),
  "vm9"= c(0,35),
  "vm12"= c(0,20),
  "vm13"= c(0,10),
  "vm14"= c(0,80),
  "vm15"= c(0,12),
  "vm20"= c(0,100),
  "vm22"= c(0,20),
  "vm24"= c(0,130),
  "pm39"= c(0,5),
  "pm40"= c(0,5),
  "pm41"= c(0,1),
  "pm42"= c(0,1),
  "pm43"= c(0,1),
  "pm44"= c(0,1),
  "pm45"= c(0,1),
  "pm46"= c(0,1),
  "vm263"= c(0,10),
  "vm136"= c(0,10),
  "vm132"= c(-5,5),
  "vm134"= c(0,120),
  "vm174"= c(0,12),
  "vm142"= c(0,80),
  "vm131"= c(0,80),
  "vm176"= c(0,40),
  "vm161"= c(0,200),
  "vm138"= c(7,8),
  "vm140"= c(0,130),
  "vm141"= c(0,100),
  "vis"= c(0,175),
  "vbpip"= c(0,70),
  "do2"= c(0,900),
  "perfpressure"= c(0,100),
  "svbpip"=  c(0,70),
  "sv"=  c(0,70),
  "svr"=  c(0,25))

VARIABLES_YRANGE_DIFF <- list (
  "vm1"= c(-3,3),
  "vm2"= c(-3,3),
  "vm3"= c(-3,3),
  "vm4"= c(-3,3),
  "vm5"= c(-3,3),
  "vm9"= c(-3,3),
  "vm12"= c(-3,3),
  "vm13"= c(-3,3),
  "vm14"= c(-3,3),
  "vm15"= c(-3,3),
  "vm20"= c(-3,3),
  "vm22"= c(-3,3),
  "vm24"= c(-3,3),
  "pm39"= c(-3,3),
  "pm40"= c(-3,3),
  "pm41"= c(-3,3),
  "pm42"= c(-3,3),
  "pm43"= c(-3,3),
  "pm44"= c(-3,3),
  "pm45"= c(-3,3),
  "pm46"= c(-3,3),
  "vm263"= c(-3,3),
  "vm136"= c(-3,3),
  "vm132"= c(-3,3),
  "vm134"= c(-3,3),
  "vm174"= c(-3,3),
  "vm142"= c(-20,20),
  "vm131"= c(-3,3),
  "vm176"= c(-3,3),
  "vm161"= c(-3,3),
  "vm138"= c(-3,3),
  "vm140"= c(-20,20),
  "vm141"= c(-3,3),
  "vis"= c(-35,35),
  "vbpip"= c(-10,10),
  "do2"= c(-100,100),
  "perfpressure"= c(-3,3),
  "svbpip"=  c(-3,3),
  "sv"=  c(-3,3),
  "svr"=  c(-3,3))

OUTPUT_PATH <- "./results/cardiac/effekt_plot"

In [3]:
config <- spark_config()
config$`sparklyr.shell.driver-memory` <- "8G"
config$`sparklyr.shell.executor-memory` <- "8G"
sc <- spark_connect(master = "local", config = config)
spark_tbl_handle <- spark_read_parquet(sc, "bolus", "/cluster/work/grlab/clinical/hirid2/research/faltysm/volume_challenge/gap_closed/psm_cardiac/2020-12-30")
df_psm <- collect(spark_tbl_handle)

Warning message in file(input, encoding = "UTF-8"):
“'raw = FALSE' but '/cluster/home/config.yml' is not a regular file”
Warning message in readLines(con):
“cannot open file '/cluster/home/config.yml': Permission denied”


In [6]:
load_single_variable <- function(variable, df_psm) { 
    df_variable = df_psm %>% select('patientid','bolusid','bolus',paste(variable, "baseline", sep="_"),contains(paste(variable, "_", sep="")))
    df_variable[,"baseline"] = df_variable[,paste(variable, "baseline", sep="_")]
    df_variable_long = gather(df_variable, condition, measurement, contains(paste(variable, "_", sep="")), factor_key=FALSE)
    df_variable_long = df_variable_long %>% separate(condition, c("variable", "timepoint"), "_")
    df_variable_long = df_variable_long %>% filter(timepoint !='trend' & timepoint != 'criterion')
    df_variable_long$uid <- with(df_variable_long, paste0(patientid, "_", bolusid))
    df_variable_long$timepoint <- gsub('baseline', '-300', df_variable_long$timepoint)
    df_variable_long$timepoint <- as.numeric(df_variable_long$timepoint)
    return (df_variable_long)
}

load_single_diff_variable <- function(variable, df_psm) {
    for (column in grep(paste(variable, "_", sep=""), names(df_psm), value=TRUE)) {
        if (column != paste(variable, "baseline", sep="_"))
            df_psm[column] <- df_psm[column]-df_psm[paste(variable, "baseline", sep="_")]
    }
    df_variable = df_psm %>% select('patientid','bolusid','bolus',contains(paste(variable, "_", sep="")))
    df_variable_long = gather(df_variable, condition, measurement, 4:last_col(), factor_key=FALSE)
    df_variable_long = df_variable_long %>% separate(condition, c("variable", "timepoint"), "_")
    df_variable_long <- df_variable_long %>% filter(timepoint !='trend' & timepoint != 'criterion' & timepoint !='baseline')
    df_variable_long$uid <- with(df_variable_long, paste0(patientid, "_", bolusid))
    df_variable_long$timepoint <- as.numeric(df_variable_long$timepoint)
    return (df_variable_long)
}

In [7]:
calc_outcome_bp_index <- function(df_total_anal)
{
    df_total_anal[,paste("vbpip", "_baseline", sep = "")] <- df_total_anal$vm5_baseline / (df_total_anal$vis_baseline+1)
    for (t in seq(0,3300,300)) {
        df_total_anal[,paste("vbpip", toString(t), sep = "_")] <- df_total_anal[,paste("vm5", toString(t), sep = "_")] / (df_total_anal[,paste("vis", toString(t), sep = "_")]+1)
    }
    return (df_total_anal)
}

calc_outcome_bp_index_strict <- function(df_total_anal)
{
    df_total_anal[,paste("svbpip", "_baseline", sep = "")] <- df_total_anal$perfpressure_baseline / (df_total_anal$vis_baseline+1)
    for (t in seq(0,3300,300)) {
        df_total_anal[,paste("svbpip", toString(t), sep = "_")] <- df_total_anal[,paste("perfpressure", toString(t), sep = "_")] / (df_total_anal[,paste("vis", toString(t), sep = "_")]+1)
    }
    return (df_total_anal)
}

calc_perf_pressure <- function(df_total_anal)
{
    df_total_anal[,paste("perfpressure", "_baseline", sep = "")] <- df_total_anal$vm5_baseline - df_total_anal$vm15_baseline
    for (t in seq(0,3300,300)) {
        df_total_anal[,paste("perfpressure", toString(t), sep = "_")] <- df_total_anal[,paste("vm5", toString(t), sep = "_")] - df_total_anal[,paste("vm15", toString(t), sep = "_")]
    }
    return (df_total_anal)
}

calc_vis <- function(df_total_anal)
{
    na_factor = 100
    a_factor = 100
    m_factor = 10
    v_factor = 10000
    d_factor = 1000 #mg compared to ug in orig. vis function
    df_total_anal[,paste("vis", "_baseline", sep = "")] <- df_total_anal$pm39_baseline * na_factor + df_total_anal$pm40_baseline * a_factor + df_total_anal$pm41_baseline * d_factor + df_total_anal$pm42_baseline * m_factor + df_total_anal$pm45_baseline * v_factor
    for (t in seq(0,3300,300)) {
        df_total_anal[,paste("vis", toString(t), sep = "_")] <- df_total_anal[,paste("pm39", toString(t), sep = "_")] * na_factor + df_total_anal[,paste("pm40", toString(t), sep = "_")] * a_factor + df_total_anal[,paste("pm41", toString(t), sep = "_")] * d_factor + df_total_anal[,paste("pm42", toString(t), sep = "_")] * m_factor + df_total_anal[,paste("pm45", toString(t), sep = "_")] * v_factor
    }
    return (df_total_anal)
}

calc_do2 <- function(df_total_anal)
{
    df_total_anal[,paste("do2", "_baseline", sep = "")] <- df_total_anal$vm13_baseline * 1.34 * df_total_anal$vm134_baseline * df_total_anal$vm141_baseline/100
    df_total_anal[,paste("do2", "_0", sep = "")] <- df_total_anal$vm13_0 * 1.34 * df_total_anal$vm134_0 * df_total_anal$vm141_0/100
    return (df_total_anal)
}

calc_sv <- function(df_total_anal)
{
    df_total_anal[,paste("sv", "_baseline", sep = "")] <- df_total_anal$vm13_baseline / df_total_anal$vm1_baseline * 1000
    for (t in seq(0,3300,300)) {
        df_total_anal[,paste("sv", toString(t), sep = "_")] <- df_total_anal[,paste("vm13", toString(t), sep = "_")] / df_total_anal[,paste("vm1", toString(t), sep = "_")] * 1000
    }
    return (df_total_anal)
}

calc_svr <- function(df_total_anal)
{
    df_total_anal[,paste("svr", "_baseline", sep = "")] <- df_total_anal$perfpressure_baseline / df_total_anal$vm13_baseline * 80
    for (t in seq(0,3300,300)) {
        df_total_anal[,paste("svr", toString(t), sep = "_")] <- df_total_anal[,paste("perfpressure", toString(t), sep = "_")] / df_total_anal[,paste("vm13", toString(t), sep = "_")] * 80
    }
    return (df_total_anal)
}

In [8]:
df_psm = calc_vis(df_psm)
df_psm = calc_outcome_bp_index(df_psm)
df_psm = calc_perf_pressure(df_psm)
df_psm = calc_outcome_bp_index_strict(df_psm)
df_psm = calc_do2(df_psm)
df_psm = calc_sv(df_psm)
df_psm = calc_svr(df_psm)

## Figure 3

In [9]:
plot_single_variable <- function(variable, df_variable_long, p_bolus, p_timepoint, p_interact) {  
    sum = df_variable_long %>%
        select(uid, timepoint, measurement, bolus) %>% #patient_id is the unique ID of the patient; moment_of_measurement is the moment of measurement; value is the value of the dependent variable; group is group
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement))
    
    y_range_size = max(abs(sum[,3:5])) * 1.2
    if (y_range_size > 10)
        y_range_size = as.integer(y_range_size)
    else
        y_range_size = as.integer(y_range_size*10)/10
    count_y_pos = y_range_size/9
    p_y_pos = y_range_size * 0.92
    plot = df_variable_long %>%
        select(uid, timepoint, measurement, bolus) %>% #patient_id is the unique ID of the patient; moment_of_measurement is the moment of measurement; value is the value of the dependent variable; group is group
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement), count = sum(!is.na(measurement))) %>%
        ggplot(aes(x = (timepoint/60) + 2.5, y = y, color = factor(bolus))) +
        geom_errorbar(aes(ymin = ymin, ymax = ymax), width = 0.5,  position = position_dodge(0.00), size = 0.5) +
        geom_point(size = 2) +
        geom_line(size = 0.5) +
        scale_color_manual(name = "", values = c("blue","red"), labels = c("Control","Bolus")) +
        ylab(VARIABLES_UNITS[variable]) +
        scale_x_continuous(expand = c(0.01, 0.01), limits=c(-5, 61)) +
        scale_y_continuous(limits=c(0, y_range_size), breaks = seq(0,y_range_size,y_range_size/10), expand=c(0,0)) +
        ggtitle(unlist(VARIABLES_CLEAR_NAMES[variable])) +
        geom_vline(xintercept=0, linetype="dashed", color = "darkgray", size=1.5) +
        geom_text(data = function(x) subset(x, bolus==1), aes(label = count, y = count_y_pos+y_range_size/30),  position=position_dodge(0.9), size = 3.5) +
        geom_text(data = function(x) subset(x, bolus==0), aes(label = count, y = count_y_pos),  position=position_dodge(0.9), size = 3.5) +
        theme(axis.text = element_text(color = "black", size = 14), axis.title = element_text(colour = "black", size = 15), panel.grid.major.y = element_line(colour = "gray"), panel.grid.minor = element_blank(), panel.background = element_blank(), axis.line = element_line(colour = "black"), legend.key=element_blank(), legend.background=element_blank(), legend.text=element_text(size=13), legend.position = c(0.20, 0.95), plot.title = element_text(hjust = 0.5, face="bold", size = 15)) +
        theme(axis.title.x=element_blank(),axis.text.x=element_blank(),axis.ticks.x=element_blank())  +              
        annotate(geom="text", x=30, y=p_y_pos+y_range_size/25, label=paste0("p (bolus): ", sprintf("%.4f",p_bolus)), color="black", size=5, hjust = 0)+
        annotate(geom="text", x=30, y=p_y_pos, label=paste0("p (interaction): ", sprintf("%.4f",p_interact)), color="black", size=5, hjust = 0)+
        annotate(geom = "text", x = -2, y = count_y_pos+y_range_size/30*3.8, label = "Pre bolus", color = "darkgray", hjust = 0, angle = 90, size=5)+
        annotate(geom = "text", x = 2, y = count_y_pos+y_range_size/30*3.8, label = "Post bolus", color = "darkgray", hjust = 0, angle = 90, size=5)
    if (variable == 'svr')
        plot <- plot + ylab(expression(dynes/seconds/cm^{-5}))
        
    return (plot)
}

plot_single_variable_diff <- function(variable, df_variable_long_diff, p_bolus=0, p_timepoint=0, p_interact=0) {  
    sum = df_variable_long_diff %>%
        select(uid, timepoint, measurement, bolus) %>% #patient_id is the unique ID of the patient; moment_of_measurement is the moment of measurement; value is the value of the dependent variable; group is group
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement))
    print(sum)
    y_range_size = max(abs(sum[,3:5])) * 2
    y_range_size = as.integer(10*y_range_size)/10
    print(y_range_size)
    count_y_pos = -y_range_size/4*3
    plot = df_variable_long_diff %>%
        select(uid, timepoint, measurement, bolus) %>% #patient_id is the unique ID of the patient; moment_of_measurement is the moment of measurement; value is the value of the dependent variable; group is group
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement), count = sum(!is.na(measurement))) %>%
        ggplot(aes(x = (timepoint/60) + 2.5, y = y, color = factor(bolus))) +
        geom_errorbar(aes(ymin = ymin, ymax = ymax), width = 0.5,  position = position_dodge(0.00), size = 0.5) +
        geom_point(size = 2) +
        geom_line(size = 0.5) +
        scale_color_manual(name = "", values = c("blue","red"), labels = c("Control","Bolus")) +
        xlab("Minutes after bolus end") +
        ylab(expression(Delta))+
        scale_x_continuous(expand = c(0.01, 0.01), limits=c(-5, 61)) +
        scale_y_continuous(limits=c(-y_range_size, y_range_size), breaks = seq(-y_range_size, y_range_size,y_range_size/2), expand=c(0,0)) +
        geom_text(data = function(x) subset(x, bolus==1), aes(label = count, y = count_y_pos+y_range_size/7),  position=position_dodge(0.9), size = 3.5) +
        geom_text(data = function(x) subset(x, bolus==0), aes(label = count, y = count_y_pos),  position=position_dodge(0.9), size = 3.5) +
        theme(legend.position = "none", panel.grid.major.y = element_line(colour = "gray"), panel.grid.minor = element_blank(), panel.background = element_blank(), axis.line = element_line(colour = "black"), axis.title = element_text(colour = "black", size = 15), axis.text = element_text(color = "black", size = 14), legend.key=element_blank(), legend.background=element_blank(), legend.text=element_text(size=13), plot.title = element_text(hjust = 0.5, face="bold", size = 15))      
    return (plot)
}
                  
#Line Plot#
plot_single_onelevel_variable <- function(variable, df_variable_long, p_bolus) {  
    y_range_size = unlist(VARIABLES_YRANGE[variable])[2]-unlist(VARIABLES_YRANGE[variable])[1]
    count_y_pos = unlist(VARIABLES_YRANGE[variable])[1] + y_range_size/10
    p_y_pos = unlist(VARIABLES_YRANGE[variable])[2] - y_range_size/8
    plot = df_variable_long %>%
        select(uid, timepoint, measurement, bolus) %>% #patient_id is the unique ID of the patient; moment_of_measurement is the moment of measurement; value is the value of the dependent variable; group is group
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement), count = sum(!is.na(measurement))) %>%
        ggplot(aes(x = (timepoint/60) + 2.5, y = y, color = factor(bolus))) +
        geom_errorbar(aes(ymin = ymin, ymax = ymax), width = 0.5,  position = position_dodge(0.00), size = 0.5) +
        geom_point(size = 2) +
        geom_line(size = 0.5) +
        scale_color_manual(name = "", values = c("blue","red"), labels = c("Control","Bolus")) +
        ggtitle("Values before and after bolus") +
        xlab("Minutes after bolus end") +
        ylab(VARIABLES_UNITS[variable]) +
        scale_x_continuous(expand = c(0.01, 0.01)) +
        scale_y_continuous(limits=c(unlist(VARIABLES_YRANGE[variable])[1], unlist(VARIABLES_YRANGE[variable])[2]), breaks = seq(unlist(VARIABLES_YRANGE[variable])[1],unlist(VARIABLES_YRANGE[variable])[2],y_range_size/10), expand=c(0,0)) +
        geom_vline(xintercept=0, linetype="dashed", color = "grey", size=1.5) +
        geom_text(data = function(x) subset(x, bolus==1), aes(label = count, y = count_y_pos+y_range_size/50),  position=position_dodge(0.9), size = 3.5) +
        geom_text(data = function(x) subset(x, bolus==0), aes(label = count, y = count_y_pos),  position=position_dodge(0.9), size = 3.5) +
        theme(panel.grid.major.y = element_line(colour = "gray"), panel.grid.minor = element_blank(), panel.background = element_blank(), axis.line = element_line(colour = "black"), axis.title = element_text(colour = "black", size = 15), axis.text = element_text(color = "black", size = 13), legend.key=element_blank(), legend.background=element_blank(), legend.text=element_text(size=13), legend.position = c(0.20, 0.95), plot.title = element_text(hjust = 0.5, face="bold", size = 15)) +
        xlim(c(-4,48)) +                  
        annotate(geom="text", x=1, y=p_y_pos, label=paste0("Bolus p: ", sprintf("%.4f",p_bolus)), color="black")
    return (plot)
}

plot_single_onelevel_variable_diff <- function(variable, df_variable_long_diff, p_bolus=0) {  
    y_range_size = unlist(VARIABLES_YRANGE_DIFF[variable])[2]-unlist(VARIABLES_YRANGE_DIFF[variable])[1]
    count_y_pos = unlist(VARIABLES_YRANGE_DIFF[variable])[1] + y_range_size/10
    p_y_pos = unlist(VARIABLES_YRANGE_DIFF[variable])[2] - y_range_size/10
    plot = df_variable_long_diff %>%
        select(uid, timepoint, measurement, bolus) %>% #patient_id is the unique ID of the patient; moment_of_measurement is the moment of measurement; value is the value of the dependent variable; group is group
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement), count = sum(!is.na(measurement))) %>%
        ggplot(aes(x = (timepoint/60) + 2.5, y = y, color = factor(bolus))) +
        geom_errorbar(aes(ymin = ymin, ymax = ymax), width = 0.5,  position = position_dodge(0.00), size = 0.5) +
        geom_point(size = 2) +
        geom_line(size = 0.5) +
        scale_color_manual(name = "", values = c("blue","red")) +
        ggtitle("Changes after bolus") +
        xlab("Minutes after bolus end") +
        ylab(expression(Delta))+
        scale_y_continuous(limits=c(unlist(VARIABLES_YRANGE_DIFF[variable])[1], unlist(VARIABLES_YRANGE_DIFF[variable])[2]), breaks = seq(unlist(VARIABLES_YRANGE_DIFF[variable])[1],unlist(VARIABLES_YRANGE_DIFF[variable])[2],y_range_size/20), expand=c(0,0)) +
        geom_text(data = function(x) subset(x, bolus==1), aes(label = count, y = count_y_pos+y_range_size/50),  position=position_dodge(0.9), size = 3.5) +
        geom_text(data = function(x) subset(x, bolus==0), aes(label = count, y = count_y_pos),  position=position_dodge(0.9), size = 3.5) +
        theme(legend.position = "none", panel.grid.major.y = element_line(colour = "gray"), panel.grid.minor = element_blank(), panel.background = element_blank(), axis.line = element_line(colour = "black"), axis.title = element_text(colour = "black", size = 15), axis.text = element_text(color = "black", size = 13), legend.key=element_blank(), legend.background=element_blank(), legend.text=element_text(size=13), plot.title = element_text(hjust = 0.5, face="bold", size = 15))
        
        #sig. of diff is depreciated
        #annotate(geom="text", x=2.75, y=p_y_pos, label=paste0("Wilcox p: ", sprintf("%.4f",p_bolus)), color="black")             
    return (plot)
}

                  
derive_lmem <- function(df_variable_long) { #currently not used
    f <- measurement ~ factor(bolus) + timepoint + (1 | uid)
    model <- lmer(f, data = df_variable_long)
    summary(model)$coefficients
}
                  
derive_lmem_baseline <- function(df_variable_long) {
    f <- measurement ~ baseline + factor(bolus) * timepoint + (1 | uid)
    model <- lmer(f, data = df_variable_long)
    summary(model)$coefficients
}
                  
derive_lmem_baseline_singular <- function(df_variable_long) {
    f <- measurement ~ baseline + factor(bolus) + (1 | uid)
    model <- lmer(f, data = df_variable_long)
    summary(model)$coefficients
}

In [10]:
plot_vars = c('vm13', 'sv', 'vm1', 'vm5', 'vis', 'svr', 'vm3', 'vm15', 'vm14')
main_plots <- list()
diff_plots <- list()
for (variable in plot_vars) {
    plots <- list()
    df_variable_data <- load_single_variable(variable, df_psm)
    df_variable_diff_data <- load_single_diff_variable(variable, df_psm)
    print(variable)
    print(summary(df_variable_diff_data))
    
    if (length(unique(df_variable_diff_data$timepoint))>=2) {
        model_coef_baseline = derive_lmem_baseline(df_variable_data)
        p1 = plot_single_variable(variable, df_variable_data,model_coef_baseline[3,5],model_coef_baseline[4,5],model_coef_baseline[5,5])
        p2 = plot_single_variable_diff(variable, df_variable_diff_data)
    } else {
        model_coef_baseline = derive_lmem_baseline_singular(df_variable_data)
        print(model_coef_baseline)
        p1 = plot_single_onelevel_variable(variable, df_variable_data,model_coef_baseline[3,5])
        p2 = plot_single_onelevel_variable_diff(variable, df_variable_diff_data)
    }
    print (model_coef_baseline)
    main_plots[[variable]] <- p1
    diff_plots[[variable]] <- p2
}

[1] "vm13"
   patientid         bolusid           bolus       variable        
 Min.   : 12462   Min.   : 1.000   Min.   :0.0   Length:85728      
 1st Qu.: 46055   1st Qu.: 1.000   1st Qu.:0.0   Class :character  
 Median : 63317   Median : 2.000   Median :0.5   Mode  :character  
 Mean   : 65310   Mean   : 2.295   Mean   :0.5                     
 3rd Qu.: 91975   3rd Qu.: 3.000   3rd Qu.:1.0                     
 Max.   :112983   Max.   :23.000   Max.   :1.0                     
                                                                   
   timepoint     measurement        uid           
 Min.   :   0   Min.   :-2.77   Length:85728      
 1st Qu.: 825   1st Qu.:-0.27   Class :character  
 Median :1650   Median : 0.05   Mode  :character  
 Mean   :1650   Mean   : 0.08                     
 3rd Qu.:2475   3rd Qu.: 0.40                     
 Max.   :3300   Max.   : 5.03                     
                NA's   :67899                     


Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”
`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)

`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)

`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



# A tibble: 24 x 5
# Groups:   timepoint [12]
   timepoint bolus        y    ymin   ymax
       <dbl> <dbl>    <dbl>   <dbl>  <dbl>
 1         0     0  0.0192  -0.0178 0.0562
 2         0     1  0.0170  -0.0246 0.0586
 3       300     0  0.0130  -0.0254 0.0514
 4       300     1  0.0555   0.0129 0.0982
 5       600     0 -0.00108 -0.0416 0.0394
 6       600     1  0.120    0.0741 0.166 
 7       900     0 -0.00564 -0.0474 0.0361
 8       900     1  0.169    0.123  0.216 
 9      1200     0  0.00320 -0.0387 0.0451
10      1200     1  0.186    0.140  0.232 
# … with 14 more rows
[1] 0.4


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



                              Estimate   Std. Error        df    t value
(Intercept)               7.363256e-01 4.196782e-02  3060.866 17.5450058
baseline                  8.453001e-01 8.472568e-03  3196.558 99.7690579
factor(bolus)1            3.042537e-02 1.428183e-02 17364.175  2.1303557
timepoint                -2.402422e-06 3.903899e-06 18042.335 -0.6153903
factor(bolus)1:timepoint  4.928834e-05 5.487936e-06 18017.954  8.9812169
                             Pr(>|t|)
(Intercept)              9.658444e-66
baseline                 0.000000e+00
factor(bolus)1           3.315628e-02
timepoint                5.383047e-01
factor(bolus)1:timepoint 2.936609e-19
[1] "sv"
   patientid         bolusid           bolus       variable        
 Min.   : 12462   Min.   : 1.000   Min.   :0.0   Length:85728      
 1st Qu.: 46055   1st Qu.: 1.000   1st Qu.:0.0   Class :character  
 Median : 63317   Median : 2.000   Median :0.5   Mode  :character  
 Mean   : 65310   Mean   : 2.295   Mean   :0.5       

...

                              Estimate   Std. Error       df     t value
(Intercept)               2.616261e+00 5.107588e-02 16925.55  51.2230257
baseline                  6.910142e-01 5.122711e-03 19395.48 134.8922736
factor(bolus)1            4.224407e-01 2.844840e-02 90184.21  14.8493671
timepoint                 2.272753e-06 8.772761e-06 86148.97   0.2590693
factor(bolus)1:timepoint -6.300700e-05 1.240868e-05 86154.73  -5.0776549
                             Pr(>|t|)
(Intercept)              0.000000e+00
baseline                 0.000000e+00
factor(bolus)1           8.044223e-50
timepoint                7.955824e-01
factor(bolus)1:timepoint 3.829160e-07
[1] "vm14"
   patientid         bolusid           bolus       variable        
 Min.   : 12462   Min.   : 1.000   Min.   :0.0   Length:85728      
 1st Qu.: 46055   1st Qu.: 1.000   1st Qu.:0.0   Class :character  
 Median : 63317   Median : 2.000   Median :0.5   Mode  :character  
 Mean   : 65310   Mean   : 2.295   Mean   :0.5     

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”
`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)

`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)

`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



# A tibble: 24 x 5
# Groups:   timepoint [12]
   timepoint bolus     y    ymin  ymax
       <dbl> <dbl> <dbl>   <dbl> <dbl>
 1         0     0 0.171 -0.0942 0.437
 2         0     1 0.820  0.514  1.13 
 3       300     0 0.101 -0.186  0.388
 4       300     1 0.914  0.585  1.24 
 5       600     0 0.109 -0.192  0.409
 6       600     1 0.735  0.413  1.06 
 7       900     0 0.135 -0.175  0.445
 8       900     1 0.656  0.326  0.985
 9      1200     0 0.120 -0.190  0.430
10      1200     1 0.586  0.243  0.929
# … with 14 more rows
[1] 2.4


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



                              Estimate   Std. Error        df   t value
(Intercept)               1.302671e+01 5.233706e-01  5428.345 24.890038
baseline                  7.892342e-01 8.338510e-03  5735.471 94.649311
factor(bolus)1            3.690985e-01 1.034571e-01 18355.651  3.567650
timepoint                -5.944152e-05 2.799819e-05 18115.312 -2.123049
factor(bolus)1:timepoint  8.067451e-05 3.932937e-05 18092.424  2.051253
                              Pr(>|t|)
(Intercept)              1.383908e-129
baseline                  0.000000e+00
factor(bolus)1            3.611151e-04
timepoint                 3.376330e-02
factor(bolus)1:timepoint  4.025665e-02


In [11]:
png(paste0(OUTPUT_PATH, "/", paste0("figure3.png")), height = 1920, width = 1620)   #pdf("path/to/file/PDFofG.pdf", height = 11, width = 8.5, paper = "letter")
ggarrange(main_plots[[plot_vars[1]]], main_plots[[plot_vars[2]]], main_plots[[plot_vars[3]]], diff_plots[[plot_vars[1]]], diff_plots[[plot_vars[2]]], diff_plots[[plot_vars[3]]], main_plots[[plot_vars[4]]], main_plots[[plot_vars[5]]], main_plots[[plot_vars[6]]], diff_plots[[plot_vars[4]]], diff_plots[[plot_vars[5]]], diff_plots[[plot_vars[6]]], main_plots[[plot_vars[7]]], main_plots[[plot_vars[8]]], main_plots[[plot_vars[9]]], diff_plots[[plot_vars[7]]], diff_plots[[plot_vars[8]]], diff_plots[[plot_vars[9]]], ncol=3, nrow=6, heights=c(3,1,3,1,3,1))
dev.off()

png 
  2

## Table 2, Supp Table 1

In [159]:
table_vars = c('vm1', 'vm2', 'vm3', 'vm4', 'vm5', 'vm9', 'vm12', 'vm13', 'vm14', 'vm15', 'vm20', 'vm22', 'vm24', 'vm263', 'vm136', 'vm132', 'vm134', 'vm174', 'vm142', 'vm138', 'vm140', 'vm141', 'pm39', 'pm40', 'pm41', 'pm42', 'pm43', 'pm44', 'pm45', 'vis', 'vbpip', 'do2', 'perfpressure', 'svbpip', 'sv', 'svr')
output <- matrix(nrow=length(table_vars)*2, ncol=16 + 4)

i=1
for (variable in table_vars) {
    print (VARIABLES_CLEAR_NAMES[variable])
    df_variable_data <- load_single_variable(variable, df_psm)
    
    #calculate model
    if (length(unique(df_variable_data$timepoint))>2) {
        model_coef_baseline = derive_lmem_baseline(df_variable_data)
    } else {
        model_coef_baseline = derive_lmem_baseline_singular(df_variable_data)
        df_psm[,'diff'] <- df_psm[paste(variable, "0", sep="_")]-df_psm[paste(variable, "baseline", sep="_")]
        print(wilcox.test(diff ~ bolus, data = df_psm))
    }
    print(model_coef_baseline)
    output[i,1] = unlist(VARIABLES_CLEAR_NAMES[variable])
    output[i,2] = paste0("p (bolus): ", sprintf("%.4f",model_coef_baseline[3,5])) 
    if (length(unique(df_variable_data$timepoint))>2)
        output[i,2] = paste(output[i,2], paste0("p (interaction): ", sprintf("%.4f",model_coef_baseline[5,5])), sep="///")  #/// will be processed to new line in excel
    output[i,3] = "Bolus"
    output[i+1,3] = "Control"
    sum = df_variable_data %>%
        select(uid, timepoint, measurement, bolus) %>%
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement), count = sum(!is.na(measurement)))
    for (r in 1:nrow(sum))
        output[i + 1 - as.integer(sum[[r,2]]), 2 + as.integer(ceiling(r/2))] <- paste(format(round(sum[[r,3]], 2), nsmall = 2), paste(paste0("(", format(round(sum[[r,4]], 2), nsmall = 2)), paste0(format(round(sum[[r,5]], 2), nsmall = 2), ")"), sep= "-"), sep=" ")
    
    output[i, 17] = which.max((sum[sum["bolus"]==1 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==1 & sum["timepoint"]== -300,'y'][[1]])[,1])
    output[i, 18] = max((sum[sum["bolus"]==1 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==1 & sum["timepoint"]== -300,'y'][[1]])[,1])
    output[i, 19] = which.min((sum[sum["bolus"]==1 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==1 & sum["timepoint"]== -300,'y'][[1]])[,1])
    output[i, 20] = min((sum[sum["bolus"]==1 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==1 & sum["timepoint"]== -300,'y'][[1]])[,1])
    output[i+1, 17] = which.max((sum[sum["bolus"]==0 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==0 & sum["timepoint"]== -300,'y'][[1]])[,1])
    output[i+1, 18] = max((sum[sum["bolus"]==0 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==0 & sum["timepoint"]== -300,'y'][[1]])[,1])
    output[i+1, 19] = which.min((sum[sum["bolus"]==0 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==0 & sum["timepoint"]== -300,'y'][[1]])[,1])
    output[i+1, 20] = min((sum[sum["bolus"]==0 & sum["timepoint"]>= 0,'y'] - sum[sum["bolus"]==0 & sum["timepoint"]== -300,'y'][[1]])[,1])
    i = i + 2
}

print (output)

$vm1
[1] "Heart rate (HR)"



Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”


                              Estimate   Std. Error       df    t value
(Intercept)               1.718891e+01 2.872386e-01 31433.55  59.841929
baseline                  8.092845e-01 3.105479e-03 33188.15 260.598922
factor(bolus)1           -1.651125e-01 4.242735e-02 92788.86  -3.891652
timepoint                -6.567441e-05 1.272680e-05 86577.75  -5.160325
factor(bolus)1:timepoint  1.284318e-04 1.799841e-05 86577.75   7.135729
                             Pr(>|t|)
(Intercept)              0.000000e+00
baseline                 0.000000e+00
factor(bolus)1           9.963341e-05
timepoint                2.470636e-07
factor(bolus)1:timepoint 9.702684e-13


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



$vm2
[1] "Temperature (central)"



...

$svr
[1] "Systemic vascular resistance (SVR)"



Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”


                              Estimate  Std. Error        df   t value
(Intercept)              196.881951172 9.640852213  3037.892 20.421634
baseline                   0.802867135 0.009069667  3038.142 88.522231
factor(bolus)1           -17.966562667 4.494814018 15659.733 -3.997176
timepoint                  0.003161462 0.001270495 18032.516  2.488370
factor(bolus)1:timepoint  -0.009321077 0.001786206 18004.777 -5.218367
                             Pr(>|t|)
(Intercept)              5.739812e-87
baseline                 0.000000e+00
factor(bolus)1           6.439639e-05
timepoint                1.284198e-02
factor(bolus)1:timepoint 1.825094e-07


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



      [,1]                                     
 [1,] "Heart rate (HR)"                        
 [2,] NA                                       
 [3,] "Temperature (central)"                  
 [4,] NA                                       
 [5,] "Systolic arterial blood pressure (SBP)" 
 [6,] NA                                       
 [7,] "DBP"                                    
 [8,] NA                                       
 [9,] "Mean arterial blood pressure (MAP)"     
[10,] NA                                       
[11,] "Mean pulmonary arterial pressure (PAPm)"
[12,] NA                                       
[13,] "PCWP"                                   
[14,] NA                                       
[15,] "Cardiac output (CO)"                    
[16,] NA                                       
[17,] "Mixed venous oxygen saturation (SvO2)"  
[18,] NA                                       
[19,] "Central venous pressure (CVP)"          
[20,] NA                                

In [160]:
if (ENDPOINT == "cardiac") {
    output_file <- "./results/cardiac/table2.csv"
}

write.csv(as.data.frame(output), file = output_file)

In [161]:
table_vars = c('vm1', 'vm2', 'vm3', 'vm4', 'vm5', 'vm9', 'vm12', 'vm13', 'vm14', 'vm15', 'vm20', 'vm22', 'vm24', 'vm263', 'vm136', 'vm132', 'vm134', 'vm174', 'vm142', 'vm138', 'vm140', 'vm141', 'pm39', 'pm40', 'pm41', 'pm42', 'pm43', 'pm44', 'pm45', 'vis', 'vbpip', 'do2', 'perfpressure', 'svbpip', 'sv', 'svr')
output <- matrix(nrow=length(table_vars)*2, ncol=15 + 4)

i=1
for (variable in table_vars) {
    print (variable)
    df_variable_data <- load_single_diff_variable(variable, df_psm)
    
    output[i,1] = unlist(VARIABLES_CLEAR_NAMES[variable])
    output[i,2] = "Bolus"
    output[i+1,2] = "Control"
    sum = df_variable_data %>%
        select(uid, timepoint, measurement, bolus) %>%
        group_by(timepoint, bolus) %>%
        summarise(mean_cl_normal(measurement), count = sum(!is.na(measurement)))
    for (r in 1:nrow(sum))
        output[i + 1 - as.integer(sum[[r,2]]), 2 + as.integer(ceiling(r/2))] <- paste(format(round(sum[[r,3]], 2), nsmall = 2), paste(paste0("(", format(round(sum[[r,4]], 2), nsmall = 2)), paste0(format(round(sum[[r,5]], 2), nsmall = 2), ")"), sep= "-"), sep=" ")
    
    output[i, 16] = which.max(unlist(sum[sum["bolus"]==1,'y']))
    output[i, 17] = max((sum[sum["bolus"]==1,'y'])[,1])
    output[i, 18] = which.min(unlist(sum[sum["bolus"]==1,'y']))
    output[i, 19] = min((sum[sum["bolus"]==1,'y'])[,1])
    output[i+1, 16] = which.max(unlist(sum[sum["bolus"]==0,'y']))
    output[i+1, 17] = max((sum[sum["bolus"]==0,'y'])[,1])
    output[i+1, 18] = which.min(unlist(sum[sum["bolus"]==0,'y']))
    output[i+1, 19] = min((sum[sum["bolus"]==0,'y'])[,1])
    i = i + 2
}

print (output)

[1] "vm1"


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



[1] "vm2"


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



[1] "vm3"


...

[1] "sv"


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



[1] "svr"


`summarise()` regrouping output by 'timepoint' (override with `.groups` argument)



      [,1]                                      [,2]     
 [1,] "Heart rate (HR)"                         "Bolus"  
 [2,] NA                                        "Control"
 [3,] "Temperature (central)"                   "Bolus"  
 [4,] NA                                        "Control"
 [5,] "Systolic arterial blood pressure (SBP)"  "Bolus"  
 [6,] NA                                        "Control"
 [7,] "DBP"                                     "Bolus"  
 [8,] NA                                        "Control"
 [9,] "Mean arterial blood pressure (MAP)"      "Bolus"  
[10,] NA                                        "Control"
[11,] "Mean pulmonary arterial pressure (PAPm)" "Bolus"  
[12,] NA                                        "Control"
[13,] "PCWP"                                    "Bolus"  
[14,] NA                                        "Control"
[15,] "Cardiac output (CO)"                     "Bolus"  
[16,] NA                                        "Control"
[17,] "Mixed v

In [162]:
if (ENDPOINT == "cardiac") {
    output_file <- "./results/cardiac/table2_diff.csv"
}

write.csv(as.data.frame(output), file = output_file)

# Responders

In [14]:
df_psm$bolus_rate = df_psm$fluidamount / (df_psm$lenght * 5) * 60 / 100
df_psm = calc_vis(df_psm)
df_psm = calc_sv(df_psm)
df_psm$fluidamount100 = df_psm$fluidamount/100

calc_change_summary <- function(df_psm, column, column_name)
{
    print(column)
    #calculate maximum and median change (positive is increase)
    df_psm[,paste("max", column_name, sep="_")] = apply(df_psm[,c(paste(column, "0", sep="_"), paste(column, "300", sep="_"), paste(column, "600", sep="_"), paste(column, "900", sep="_"), paste(column, "1200", sep="_"), paste(column, "1500", sep="_"), paste(column, "1800", sep="_"), paste(column, "2100", sep="_"), paste(column, "2400", sep="_"), paste(column, "2700", sep="_"), paste(column, "3000", sep="_"), paste(column, "3300", sep="_"))], 1, max, na.rm=TRUE) - df_psm[,paste(column, "baseline", sep="_")]
    df_psm[,paste("median", column_name, sep="_")] = apply(df_psm[,c(paste(column, "0", sep="_"), paste(column, "300", sep="_"), paste(column, "600", sep="_"), paste(column, "900", sep="_"), paste(column, "1200", sep="_"), paste(column, "1500", sep="_"), paste(column, "1800", sep="_"), paste(column, "2100", sep="_"), paste(column,"2400", sep="_"), paste(column, "2700", sep="_"), paste(column, "3000", sep="_"), paste(column, "3300", sep="_"))], 1, median, na.rm=TRUE) - df_psm[,paste(column, "baseline", sep="_")]
    
    #set ratio to 1 if baseline value 0 and outcome values > 0
    baseline_zero_or_nd = df_psm[,paste(column, "baseline", sep="_")] == 0 | is.na(df_psm[,paste(column, "baseline", sep="_")])
    baseline_zero = df_psm[,paste(column, "baseline", sep="_")] == 0
    max_nd = is.na(df_psm[,paste("max", column, sep="_")])
    max_zero = df_psm[,paste("max", column, sep="_")] == 0
    
    #default values if one is missing
    df_psm[max_nd | max_zero | baseline_zero_or_nd, paste("ratio", column_name, sep="_")] = as.double(NA)
    if (column=="vis") {
        #special case of start of vasoactiva
        df_psm[!max_zero & baseline_zero_or_nd, paste("ratio", column_name, sep="_")] = 1.0
        df_psm[max_zero & baseline_zero_or_nd, paste("ratio", column_name, sep="_")] = 0.0
    }
    df_psm[!max_nd & !baseline_zero_or_nd, paste("ratio", column_name, sep="_")] = df_psm[!max_nd & !baseline_zero_or_nd,paste("max", column_name, sep="_")] / df_psm[!max_nd & !baseline_zero_or_nd,paste(column, "baseline", sep="_")]
    df_psm[!max_nd & !baseline_zero_or_nd, paste("ratio_bin", column_name, sep="_")] = df_psm[!max_nd & !baseline_zero_or_nd,paste("ratio", column_name, sep="_")] > 0.1
    df_psm
}

#calculate relative changes in variables
plot_vars = c('vm5', 'vis', 'vm13', 'sv')
for (variable in plot_vars){
    df_psm = calc_change_summary(df_psm, variable,variable)
}

[1] "vm5"


Warning message:
“The `i` argument of ``[<-`()` can't be a matrix as of tibble 3.0.0.
Convert to a vector.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


[1] "vis"
[1] "vm13"


Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning me

Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning me

...

Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning me

Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning me

Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning me

Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning me

Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”
Warning me

In [15]:
calc_ratio_of_responders <- function(df_psm, column)
{
    print (column)
    resp_bolus = sum(df_psm[df_psm$bolus==1,column] > 0.1, na.rm=TRUE)
    resp_controls = sum(df_psm[df_psm$bolus==0,column] > 0.1, na.rm=TRUE)
    total_bolus = sum(!is.na(df_psm[df_psm$bolus==0,column]), na.rm=TRUE)
    total_controls = sum(!is.na(df_psm[df_psm$bolus==1,column]), na.rm=TRUE)
    res = prop.test(x = c(resp_bolus, resp_controls), n = c(total_bolus, total_controls))
    c(resp_bolus/total_bolus, resp_controls/total_controls, res$p.value)
}

calc_ratio_of_responders(df_psm, 'ratio_vm5')
calc_ratio_of_responders(df_psm, 'ratio_vm13')
calc_ratio_of_responders(df_psm, 'ratio_sv')
calc_ratio_of_responders(df_psm, 'ratio_vis')

[1] "ratio_vm5"


[1] 0.5165174 0.5338746 0.1483893

[1] "ratio_vm13"


[1] 6.064935e-01 4.909091e-01 6.596079e-06

[1] "ratio_sv"


[1] 6.233766e-01 5.051948e-01 3.742139e-06

[1] "ratio_vis"


[1] 8.902576e-02 5.123180e-02 5.355188e-10

In [16]:
#vis increase in map responsers (bolus)
ct = table(df_psm[df_psm$bolus==1,'ratio_vis'] > 0.1, df_psm[df_psm$bolus==1,'ratio_vm5'] > 0.1,dnn=list("VIS","MAP"))
print (ct)
print (ct[2,2]/(ct[1,2] + ct[2,2]))

       MAP
VIS     FALSE TRUE
  FALSE  1604 1650
  TRUE    123  195
[1] 0.1056911


In [17]:
#vis increase in map responsers (controls)
ct = table(df_psm[df_psm$bolus==0,'ratio_vis'] > 0.1, df_psm[df_psm$bolus==0,'ratio_vm5'] > 0.1,dnn=list("VIS","MAP"))
print (ct)
print (ct[2,2]/(ct[1,2] + ct[2,2]))

       MAP
VIS     FALSE TRUE
  FALSE  1598 1791
  TRUE     67  116
[1] 0.06082853
